# Table of Contents
 <p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Citation network of drafts of Chinese constitution

Hive Plot visualization

In [1]:
from pyveplot import Hiveplot, Axis, Node
import networkx as nx
import csv

G = nx.DiGraph()

with open('nodes.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for i, data in enumerate(readCSV):
        if data[0] != "id":
            G.add_node(data[0], idx=i, axis=data[1], label=data[2], pos=data[3], height=data[4])

with open('edges_v2.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        if row[0] != "source":
            G.add_edge(row[0], row[1])

ModuleNotFoundError: No module named 'pyveplot'

In [ ]:
G.nodes.data()

In [ ]:
colors = {}
colors[0] = ['#aae890', # bright green
             '#4daf4a', # apple green 
             '#72b077'  # greyish green
            ]
colors[1] = ['#e41a1c', # red
             '#377eb8', # cold blue
             '#984ea3', # purple
             '#ff7f00', # orange
             '#a65628', # brown
             '#f781bf'  # pink
            ]
colors[2] = ['#ffff33'] # bright yellow


# create hiveplot object
h = Hiveplot('hiveplot.svg')

# create three axes, spaced at 120 degrees from each other
h.axes = [Axis(start=20, angle=-90,
               stroke=colors[0][0], stroke_width=1),
          Axis(start=20, angle=-90 + 120,
               stroke=colors[1][0], stroke_width=1),
          Axis(start=20, angle=-90 + 120 + 120,
               stroke=colors[2][0], stroke_width=1)
          ]

# categorize them into three groups
draft_articles = [v for v in G.nodes() if G.nodes[v]['axis'] == "Draft Articles"]
foreign_leg    = [v for v in G.nodes() if G.nodes[v]['axis'] == "Foreign Legislation"]
trad_sources   = [v for v in G.nodes() if G.nodes[v]['axis'] == "Traditional Sources"]

# place these nodes onto our three axes
for axis, nodes in zip(h.axes,
                       [draft_articles, foreign_leg, trad_sources]):
    s = ""
    color_idx = -1
    circle_color = colors[2][0]
    for v in nodes:
        if axis == h.axes[0] and G.nodes[v]['label'][:10] != s :
            color_idx += 1
            circle_color = colors[0][color_idx % len(colors[0])]
            s = G.nodes[v]['label'][:10]
        if axis == h.axes[1] and G.nodes[v]['label'][:4] != s :
            color_idx +=1
            circle_color = colors[1][color_idx % len(colors[1])]
            s = G.nodes[v]['label'][:4]
        # create node object
        node = Node(radius=8,
                    label="%s" % (G.nodes[v]['label']))
        # add it to axis
        axis.add_node(v, node)
        # once it has x, y coordinates, add a circle
        node.add_circle(fill=circle_color, stroke=circle_color,
                        stroke_width=0.1, fill_opacity=0.7)
        if axis.angle < 0:
            orientation = 1
            scale = 0.6
        else:
            orientation = -1
            scale = 0.35
        # also add a label
        node.add_label(label="%s" % (G.nodes[v]['label']),
                       angle=axis.angle + 90 * orientation,
                       scale=scale)

# iterate through axes, from left to right
for n in range(-1, len(h.axes) - 1):
    # draw curves between nodes connected by edges in network
    h.connect_axes(h.axes[n],
                   h.axes[n+1],
                   G.edges,
                   stroke_width=0.3,
                   stroke='#999999')

# save output
h.save('hiveplot.svg')

Next, try with graph_tool:

In [21]:
from graph_tool.all import *
import csv

hy = Graph()
v_name = hy.new_vertex_property("string")
v_color = hy.new_vertex_property("string")
v_size = hy.new_vertex_property("float")

# The file nodes.tsv contains the hierarchy of nodes, from a single "Origin" origin, in "from"-"to" relations
with open('nodes.tsv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter='\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        q  = row[0]       # from
        qc = row[1]       # from_color
        z  = row[2]       # to
        zc = row[3]       # to_color
        if len(graph_tool.util.find_vertex(hy, v_name, q)) == 0:
            s = hy.add_vertex()
            v_name[s] = q
            v_color[s] = qc
            if qc == "black": v_size[s] = 0
            else: v_size[s] = 10
        else:
            s = graph_tool.util.find_vertex(hy, v_name, q)[0]

        if len(graph_tool.util.find_vertex(hy, v_name, z)) == 0:
            t = hy.add_vertex()
            v_name[t] = z
            v_color[t] = zc
            if zc == "black": v_size[t] = 0
            else: v_size[t] = 10
        else:
            t = graph_tool.util.find_vertex(hy, v_name, z)[0]
        
        if not hy.edge(s, t):
            hy.add_edge(s, t)

pos = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

graph_draw(hy, pos=pos, vertex_text=v_name, vertex_fill_color=v_color, vertex_font_size=10, vertex_text_position="centered",
            output_size=(1024, 768), edge_pen_width=2, output="hierarchy.png")

if graph_tool.util.find_vertex(hy, "in", 0):
    l = graph_tool.util.find_vertex(hy, "in", 0)
    for n in l:
        print("Indegree problem with vertex {}".format(v_name[n]))

hy.save("hierarchy.xml.gz")

Indegree problem with vertex Origin


In [22]:
nw = Graph()
nwv_name  = nw.new_vertex_property("string")
nwv_color = nw.new_vertex_property("string")
nwv_size  = nw.new_vertex_property("float")
nwe_color = nw.new_edge_property("string")

for v in hy.vertices():
    n = nw.add_vertex()
    nwv_name[n]  = v_name[v]
    nwv_color[n] = v_color[v]
    nwv_size[n]  = v_size[v]

with open('edges_specific.tsv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter = '\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        source = ""
        target = ""
        
        f = row[0]       # from
        t = row[1]       # to
        d = row[2]       # direct
        n = row[3]       # footnote
        a = row[4]       # attitude
        s = row[5]       # specific
        c = row[6]       # e_color

        if len(graph_tool.util.find_vertex(nw, nwv_name, f)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a source in edges list.".format(f))
        else:
            source = graph_tool.util.find_vertex(nw, nwv_name, f)[0]
        
        if len(graph_tool.util.find_vertex(nw, nwv_name, t)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a target in edges list.".format(t))
        else:
            target = graph_tool.util.find_vertex(nw, nwv_name, t)[0]
        
        e = nw.add_edge(source, target)
        nwe_color[e] = c

with open('edges_unspecific.tsv') as csvfile:
    # readCSV = csv.reader(csvfile, delimiter = '\t')
    # next(readCSV, None)  # skip the headers

    for row in csv.reader(csvfile.readlines()[1:], delimiter = '\t'):
        f = row[0]       # from
        t = row[1]       # to
        d = row[2]       # direct
        n = row[3]       # footnote
        a = row[4]       # attitude
        s = row[5]       # specific
        c = row[6]       # e_color

        if len(graph_tool.util.find_vertex(nw, nwv_name, f)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a source in edges list.".format(f))
        else:
            source = graph_tool.util.find_vertex(nw, nwv_name, f)[0]
        
        if len(graph_tool.util.find_vertex(nw, nwv_name, t)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a target in edges list.".format(t))
        else:
            target = graph_tool.util.find_vertex(nw, nwv_name, t)[0]

        e = nw.add_edge(source, target)
        nwe_color[e] = c

nw.save("network.xml.gz")

graph_draw(nw, vertex_text=nwv_name, vertex_fill_color=nwv_color, vertex_font_size=10, vertex_text_position=10,
           output_size=(1024, 768), edge_pen_width=2, output="network_1.png")


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7ff1d86347c0, at 0x7ff1d8634d30>

In [29]:
# state = graph_tool.inference.minimize.minimize_nested_blockmodel_dl(nw, deg_corr=True)
# t     = graph_tool.inference.nested_blockmodel.get_hierarchy_tree(state)[0]

hypos  = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

cts   = graph_tool.draw.get_hierarchy_control_points(nw, hy, hypos, beta=0.8, max_depth=3)
pos   = nw.own_property(hypos)

graph_draw(nw, pos=pos, vertex_text=nwv_name, vertex_fill_color=nwv_color,
           vertex_font_size=nwv_size, vertex_size=nwv_size, vertex_text_position="centered",
           edge_control_points=cts, edge_pen_width=1, edge_color=nwe_color,
           output_size=(1500, 1500), output="network_2.png")


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7ff1d86347c0, at 0x7ff1d8888340>